In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
path = '/home/alvaro/Escritorio/FIUBA/Programación/Organizacion de datos'
df= pd.read_csv(path + '/train.csv')
#Fuente: https://es.investing.com/currencies/usd-mxn-historical-data
#AGREGAR EL PATH CORRESPONDIENTE
path1 = '/home/alvaro/Descargas'
dolar= pd.read_csv(path1 + '/Datos históricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)
df['fecha'] = pd.to_datetime(df['fecha'])
df['year'] = df['fecha'].dt.year
df['month']= df['fecha'].dt.month
#Agrego la columna precio en dolar por propiedad, en el dataframe original
newDf= pd.merge(df, dolar, on=['year', 'month'], how='left')
newDf["dollar_price"] = newDf.apply(lambda row: row["precio"]/row["price"], axis=1)
#Filtro dejando las cosas útiles.
#Para agregar una columna, se debe agregar aca y luego agregarla al merge
df = newDf.filter(["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                   'centroscomercialescercanos', 'precio' ,'dollar_price' ])
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales
    
#Arreglando, no pueden haber propiedades con metros cubiertos pero sin metros totales
df['metrostotales'].fillna(0, inplace=True)
df['metrostotales'] = df.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
df.head()
#Filtrando el garage, lotes, hospedaje, otros
filtroTerreno = df["tipodepropiedad"].isin(["Huerta", "Nave industrial", "Terreno", "Terreno comercial", 
                                            "Bodega comercial", "Terreno industrial"])
dfTerreno = df[filtroTerreno]
dfTerreno.tipodepropiedad.value_counts()
#Dropeo todos los terrenos que tienen habitaciones ya que si las tienen no serian terrenos.
dfTerreno = dfTerreno[dfTerreno["habitaciones"].isna()]
dfTerreno.tipodepropiedad.value_counts()
#Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
#tener metros no cubiertos
filtro1 = df["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio", "Edificio"
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
dfCasas = df[filtro1]
dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
dfCasas.garages.fillna(0, inplace=True)
df = pd.merge(dfCasas, dfTerreno, on=['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos'
                                      ,'ciudad','provincia','metrostotales','idzona','fecha', 'gimnasio','usosmultiples',
                                       'piscina','escuelascercanas','centroscomercialescercanos', 
                                      'precio', 'dollar_price'], how='outer')
df

,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,dollar_price
0,254099,Apartamento,2.0,1.0,2.0,Benito Juárez,Distrito Federal,80.0,80.0,23533.0,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,135683.671398
1,53461,Casa en condominio,3.0,2.0,2.0,La Magdalena Contreras,Distrito Federal,268.0,180.0,24514.0,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0,278078.170864
2,247984,Casa,3.0,2.0,2.0,Tonalá,Jalisco,144.0,166.0,48551.0,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0,72727.272727
3,209067,Casa,2.0,1.0,1.0,Zinacantepec,Edo. de México,63.0,67.0,53666.0,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0,50753.494183
4,185997,Apartamento,2.0,1.0,1.0,Zapopan,Jalisco,95.0,95.0,47835.0,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0,62910.284464
5,126147,Apartamento,2.0,1.0,1.0,Coyoacán,Distrito Federal,75.0,90.0,23650.0,2014-03-18,0.0,0.0,0.0,0.0,1.0,1100000.0,84240.836894
6,139233,Casa,3.0,1.0,2.0,Oaxaca de Juárez,Oaxaca,140.0,160.0,73510.0,2016-02-23,0.0,0.0,0.0,0.0,0.0,1150000.0,63413.289220
7,5013,Casa,4.0,2.0,3.0,Playa del Carmen,Quintana Roo,293.0,293.0,130510.0,2016-10-20,0.0,0.0,0.0,0.0,0.0,4200000.0,222646.310433
8,44962,Apartamento,2.0,1.0,1.0,Villa de Alvarez,Colima,58.0,58.0,9010.0,2014-01-06,0.0,0.0,0.0,1.0,1.0,310000.0,23210.194516
9,51180,Casa,3.0,1.0,1.0,San Luis Potosí,San luis Potosí,100.0,100.0,87542.0,2013-01-03,0.0,0.0,0.0,1.0,1.0,488000.0,38398.589953


In [8]:
id = df.dollar_price.idxmax(axis = 1, skipna = True) 
df.iloc[id]

id                                           197656
tipodepropiedad                                Casa
habitaciones                                      3
garages                                           0
banos                                             3
ciudad                        Cuajimalpa de Morelos
provincia                          Distrito Federal
metroscubiertos                                 422
metrostotales                                   422
idzona                                        23779
fecha                           2013-04-03 00:00:00
gimnasio                                          0
usosmultiples                                     0
piscina                                           0
escuelascercanas                                  1
centroscomercialescercanos                        1
precio                                      1.2e+07
dollar_price                                 988957
Name: 20328, dtype: object

In [9]:
id = df.precio.idxmax(axis = 1, skipna = True)
df.iloc[id]

id                                         181271
tipodepropiedad                              Casa
habitaciones                                    3
garages                                         0
banos                                           4
ciudad                                    Tlalpan
provincia                        Distrito Federal
metroscubiertos                               300
metrostotales                                 300
idzona                                      24763
fecha                         2016-10-25 00:00:00
gimnasio                                        0
usosmultiples                                   0
piscina                                         0
escuelascercanas                                1
centroscomercialescercanos                      1
precio                                 1.2521e+07
dollar_price                               663751
Name: 175405, dtype: object